#Zadanie 7 (7 pkt)
Celem zadania jest zaimplementowanie dwóch wersji naiwnego klasyfikatora Bayesa.
* W pierwszej wersji należy dokonać dyskretyzacji danych - przedział wartości każdego atrybutu dzielimy na cztery równe przedziały i każdej ciągłej wartości atrybutu przypisujemy wartość dyskretną wynikająca z przynależności do danego przedziału.
* W drugiej wersji wartości likelihood wyliczamy z rozkładów normalnych o średnich i odchyleniach standardowych wynikających z wartości atrybutów.
Trening i test należy przeprowadzić dla zbioru Iris, tak jak w przypadku zadania z drzewem klasyfikacyjnym. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania klasyfikatorów dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Dyskretyzacja danych - **0.5 pkt**
* Implementacja funkcji rozkładu normalnego o zadanej średniej i odchyleniu standardowym. - **0.5 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych dyskretnych. - **2.0 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych ciągłych. - **2.5 pkt**
* Przeprowadzenie eksperymentów, wnioski i sposób ich prezentacji. - **1.5 pkt**


In [35]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import math
from collections import Counter
from typing import List
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

print("x_train: ", x_train)
print("x_test: ", x_test)
print("y_train: ", y_train)
print("y_test: ", y_test)

x_train:  [[6.5 3.  5.8 2.2]
 [5.5 3.5 1.3 0.2]
 [4.3 3.  1.1 0.1]
 [6.1 2.9 4.7 1.4]
 [4.8 3.  1.4 0.3]
 [5.2 3.4 1.4 0.2]
 [6.3 2.8 5.1 1.5]
 [4.8 3.4 1.9 0.2]
 [6.1 3.  4.9 1.8]
 [5.1 3.8 1.6 0.2]
 [5.4 3.4 1.7 0.2]
 [5.4 3.4 1.5 0.4]
 [5.6 2.8 4.9 2. ]
 [7.7 3.8 6.7 2.2]
 [5.  3.6 1.4 0.2]
 [7.4 2.8 6.1 1.9]
 [6.  2.2 5.  1.5]
 [4.7 3.2 1.6 0.2]
 [5.1 3.5 1.4 0.2]
 [6.  2.2 4.  1. ]
 [5.  2.3 3.3 1. ]
 [7.9 3.8 6.4 2. ]
 [5.4 3.9 1.7 0.4]
 [5.4 3.9 1.3 0.4]
 [5.8 2.7 3.9 1.2]
 [5.  2.  3.5 1. ]
 [5.  3.2 1.2 0.2]
 [6.8 3.2 5.9 2.3]
 [6.7 3.  5.2 2.3]
 [5.8 2.7 5.1 1.9]
 [5.8 2.8 5.1 2.4]
 [6.3 3.4 5.6 2.4]
 [5.5 2.3 4.  1.3]
 [5.1 3.8 1.5 0.3]
 [4.4 3.  1.3 0.2]
 [6.5 3.2 5.1 2. ]
 [5.1 3.3 1.7 0.5]
 [4.9 3.1 1.5 0.1]
 [6.7 3.1 4.7 1.5]
 [6.1 3.  4.6 1.4]
 [5.5 2.5 4.  1.3]
 [5.7 2.6 3.5 1. ]
 [5.8 2.7 5.1 1.9]
 [6.7 3.1 4.4 1.4]
 [6.4 3.2 5.3 2.3]
 [4.5 2.3 1.3 0.3]
 [6.7 3.3 5.7 2.1]
 [5.7 3.  4.2 1.2]
 [5.1 3.7 1.5 0.4]
 [4.8 3.4 1.6 0.2]
 [6.3 2.9 5.6 1.8]
 [6.4 2.9 4.3 1.3]
 [

In [36]:
class BayesSkeleton(ABC):
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def _calc_priors(self, train_classes):
        total_samples = len(train_classes)
        for c in np.unique(train_classes):
            self.priors[c] = np.sum(train_classes == c) / total_samples

    def _calc_likelihoods(self, train_features, train_classes):
        for c in self.classes:
            self.likelihoods[c] = {}
            class_features = train_features[train_classes == c]

            for i in range(train_features.shape[1]):
                values, counts = np.unique(class_features[:, i], return_counts=True)

                for value, count in zip(values, counts):
                    # P(X_i = value | Y = c)
                    self.likelihoods[c][(i, value)] = count / len(class_features)

In [37]:
class NaiveBayes(BayesSkeleton):
    def __init__(self):
        super().__init__()
        self.classes: List[int] = None
        self.num_classes: int = None

        self.bins: List[np.ndarray] = None

    def build_classifier(self, train_features, train_classes):
        self._compute_bins(train_features)
        discretized_features = self._calc_discretized_features(train_features)

        self.classes = np.unique(train_classes)
        self.num_classes = len(self.classes)

        self._calc_priors(train_classes)
        self._calc_likelihoods(discretized_features, train_classes)

    @staticmethod
    def data_discretization(data, bins):
        discretized_data = np.digitize(data, bins, right=False) - 1
        return discretized_data

    def _compute_bins(self, train_features):
        self.bins = [
            np.linspace(np.min(train_features[:, i]), np.max(train_features[:, i]), 5)
            for i in range(train_features.shape[1])
        ]

    def _calc_discretized_features(self, train_features):
        return np.array(
            [
                self.data_discretization(train_features[:, i], self.bins[i])
                for i in range(train_features.shape[1])
            ]
        ).T

    def predict(self, sample):
        print(f"Sample: {sample}")
        discretized_sample = self._discretize_sample(sample)
        print(f"Discretized Sample: {discretized_sample}")
        prediction = self._predict_class(discretized_sample)
        print(f"Prediction: {prediction}")
        return prediction

    def _discretize_sample(self, sample):
        discretized_sample = np.zeros_like(sample, dtype=int)
        for i in range(len(sample)):
            discretized_sample[i] = NaiveBayes.data_discretization(sample[i], self.bins[i])

        return discretized_sample

    def _predict_class(self, discretized_sample):
        class_probs = []
        for c in self.classes:
            # Log(P(Y = c))
            class_prob_log = np.log(self.priors[c])

            for i, feature in enumerate(discretized_sample):
                # Log(P(X_i = value | Y = c))
                likelihood = self.likelihoods[c].get((i, feature), 1e-10)
                class_prob_log += np.log(likelihood)

            class_probs.append(class_prob_log)

        return self.classes[np.argmax(class_probs)]

class GaussianNaiveBayes(BayesSkeleton):
    def __init__(self):
        super().__init__()

        self.classes: List[int] = None
        self.num_classes: int = None

        self._means: np.ndarray = []
        self._std = None

    def build_classifier(self, train_features, train_classes):
        self.classes = np.unique(train_classes)
        self.num_classes = len(self.classes)

        self._calc_priors(train_classes)
        self._calc_likelihoods(train_features, train_classes)

    @staticmethod
    def normal_dist(x, mean, std):
        if std == 0:
            return 1.0 if x == mean else 0.0
        exponent = np.exp( -((x - mean) ** 2) / (2 * (std ** 2)))
        return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

    def predict(self, sample):
        pass


# Eksperymenty

In [38]:
nbc: NaiveBayes = NaiveBayes()
nbc.build_classifier(x_train, y_train)

predictions = [nbc.predict(sample) for sample in x_test]
accuracy = np.mean(predictions == y_test)

print("Accuracy: ", accuracy)

Accuracy:  0.9333333333333333
